## ライブラリのインポート

In [ ]:
from qiskit import *
from qiskit_aer import Aer
import numpy as np 
import matplotlib.pyplot as plt

import cv2

from common_logic import *
from common_quantum import *

%matplotlib inline

## プロパティ

In [ ]:
imageFileName = "test.jpg"
shot_count = 2**16

In [ ]:
image = cv2.imread(f"{imageFileName}")
gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

width = len(gray_image[0])
height = len(gray_image)
w_qubits = width.bit_length()
h_qubits = height.bit_length()
n_qubits = w_qubits + h_qubits

In [ ]:
probabilities = create_matrix(width,height)
for y in range(height):
    for x in range(width):
        probabilities[y][x] = np.sqrt(gray_image[y][x]/np.sum(gray_image))

address = gray_code_2d(width,height)

In [ ]:
state_init = convert_probability2statevector(probabilities,address)

qc = create_QuantumCircuit_by_Statevector(state_init)
backend = Aer.get_backend("qasm_simulator")
result = backend.run(qc, shots=shot_count).result()
counts = result.get_counts()

In [ ]:
image_quantumOutput = create_matrix(width,height)
for key,value in counts.items():
    x = gray_to_decimal(key[:w_qubits])
    y = gray_to_decimal(key[w_qubits:])
    val = clamp(float(np.sqrt(value*np.sum(gray_image)/(shot_count*255)))*255, 0, 255)
    if(x<width and y<height):
        image_quantumOutput[y][x] = val

cv2.imwrite('quantumOutput.png',np.array(image_quantumOutput))

In [ ]:
plt.imshow(image_quantumOutput, cmap='gray')
plt.title('Gray Image')
plt.axis('off')
plt.show()